In [1]:
# Libraries and stuff
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
import IPython
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable



In [2]:
'''
Constants for discretizing states. There are 50 states of theta and thetadot each.
State 0 corresponds to theta = -pi, thetadot = -6.5m/s
State 2499 corresponds to theta = pi, thetadot = 6.5m/s
States [24, 25, 2474 and 2475] have values that approximately equal theta = -pi , thetadot = 0 
These 4 states are considered goal states.
'''
maxVelocity = 6.5
thetaStates = 50
thetaDotStates = 50
thetaStepSize = 2*math.pi/49
thetaDotStepSize = 2*maxVelocity/49

In [3]:
def pendulum(state,action):
    
    '''
    This function takes state(number from 0 to 2499 ) and action (0, 1 or 2) and computes the next state
    of the pendulum based on the action. 
    
    '''
    #constants
    GRAVITY = 9.8
    LENGTH = 1.0
    dt1 = 0.001
    FORCE = 4.0
    time = 200
    if (action == 0):
        u = -FORCE
    elif(action == 1):
        u = FORCE
    elif(action == 2):
        u = 0
    # Get values of theta and thetadot that correspond to the state
    theta, thetaDot = get_values(state)
    
    # Do Euler update of values for theta and thetadot to calculate the new theta and thetadot 
    for i in range(time):
        
        newThetaDot = thetaDot + dt1*(-(GRAVITY/LENGTH)*np.sin(theta) + u)
        newTheta = theta + dt1*newThetaDot
        
        theta = ((newTheta + np.pi)%(2*np.pi)) - np.pi
        thetaDot = np.clip(newThetaDot, -maxVelocity, maxVelocity)
    
    #Discretize the values theta and thetadot back into a state(number from 0 to 2499)
    next_state = get_state(newTheta, newThetaDot)
    return next_state
    

In [4]:
def get_state(theta, thetaDot):
    '''
    Discretize the values theta and thetadot back into a state(number from 0 to 2499)
    '''
    state = 0
    for i in range (thetaStates):
        for j in range (thetaDotStates):
            if ((theta >= (-np.pi + thetaStepSize*i)) & (thetaDot >=(-maxVelocity + thetaDotStepSize*j))):
                state = i*thetaStates + j    
    return state

def get_values(state):
    '''
    Get values of theta and thetadot that correspond to the state
    '''
    thetaDot = -maxVelocity + (state%thetaDotStates)*thetaDotStepSize
    theta = -np.pi + int(state/thetaStates)*thetaStepSize
    return theta, thetaDot

In [5]:
def draw_pendulum(theta_array):
    '''
    This function takes an array of theta values for each time instant and creates an animation. 
    
    '''
    x1 = np.sin(theta_array)
    y1 = -np.cos(theta_array)

    x1= x1[:,0::200]
    y1= y1[:,0::200]

    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
    ax.grid()

    line, = ax.plot([], [], 'o-', lw=2)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

    dt = 0.0005
    t = np.arange(0.0, 20, dt)

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text


    def animate(i):
        thisx = [0, x1[i]]
        thisy = [0, y1[i]]

        line.set_data(thisx, thisy)
        time_text.set_text(time_template % (i*dt))
        return line, time_text

    ani = animation.FuncAnimation(fig, animate, np.arange(1, len(theta_array)),
                                  interval=200, blit=True, init_func=init)


    plt.close(fig)
    plt.close(ani._fig)
    IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))

In [6]:
# state transition table
def generate_next_state():
    
'''
This function makes a table of all possible states(rows) and 
actions(columns) and records next state according to the pendulum dynamics.  
'''
    next_states = np.zeros((thetaStates*thetaDotStates,3))
    for i in range (thetaStates*thetaDotStates):
        for j in range(3):
            next_states[i,j] = int(pendulum(i,j))
    #     print(next_states[i])
    return next_states

IndentationError: expected an indented block (<ipython-input-6-e16cb4e01766>, line 7)

In [ ]:
# A matrix with next states, next_state = next_states[state,action]. This table can be used
# to compute the next state of pendulum, given an action, at each step.
next_states = generate_next_state()
    

###  Defining Neural Network to store Q(s,a) 

In [ ]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 1, 2, 32, 32, 1

# Create empty Tensor to hold input
x = torch.empty(1, 2)

# Use the nn package to define our model and loss function.
policy_net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Linear(H1, H2),
    torch.nn.Linear(H2, D_out),)

#target_net is used to freeze the target every few steps
target_net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H1),
    torch.nn.Linear(H1, H2),
    torch.nn.Linear(H2, D_out),)


target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4
optimizer = torch.optim.Adam(policy_net.parameters(), lr=learning_rate)


In [ ]:
def run_episode():
    #hyperparameters and initial conditions for each episode
    currentState = 1225
    currentAct = 1.0
    episodeSteps = 500
    EPSILON = 0.25
    GAMMA = 0.999
    TARGET_UPDATE = 10
    Qa_values = np.zeros(3)
    Qa1_values = np.zeros(3)
    
    for i in range(episodeSteps):
        #get next state based on current state and current best action 
        nextState = next_states[int(currentState),int(currentAct)]
        
        # set the reward for current state
        if((currentState ==24)|(currentState ==25)|(currentState ==2474)|(currentState ==2475)):
            reward = 0
        else:
            reward = -1
        
        #######
        #evaluate Q(s,a) at current state and current best action from neural net
        x[0][0] = currentState
        x[0][1] = float(currentAct)
        state_action_values = policy_net(x)
        state_action_values.requires_grad_(True)
        #######
        
        #######
        #evaluate max_a(Q(s',a)) at current state and current best action from 'target' neural net 
        x[0][0] = nextState
        x[0][1] = 0
        Qa1_values[0] = target_net(x).detach()
        x[0][1] = 1
        Qa1_values[1] = target_net(x).detach()
        x[0][1] = 2
        Qa1_values[2] = target_net(x).detach()
        nextAct1 = np.argmax(Qa1_values)
        x[0][1] = float(nextAct1)
        next_state_values = target_net(x).detach()
        #######
        
        # Target value for our loss function
        expected_state_action_values = (next_state_values * GAMMA) + reward
        expected_state_action_values.requires_grad_(True)
        
        # Calculate loss
        loss = loss_fn(state_action_values, expected_state_action_values)
        loss = Variable(loss, requires_grad = True)
        
        # Update policy
        policy_net.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        ########
        # Best next action for next state, evaluated from our model, policy_net. 
        if (EPSILON > random.randint(1,10)*0.1):
            nextAct = np.array(random.randint(0,2))
        else:
            x[0][0] = nextState
            
            x[0][1] = 0
            Qa_values[0] = policy_net(x).detach()


            x[0][1] = 1
            Qa_values[1] = policy_net(x).detach()


            x[0][1] = 2
            Qa_values[2] = policy_net(x).detach()

            nextAct = np.argmax(Qa_values)
        ########
        
        
        #update state
        currentState = nextState
        currentAct = nextAct
        # Update the target network, copying all weights and biases in policy_net
        if i % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())
            

In [ ]:
# Run episodes  
for i in range(1000):
    run_episode()


In [ ]:
# Here I'm trying to get a sequence of actions from the model that has already learned.
# Start at the pendulum down position at state = 1225
St = 1224
NextValue = np.zeros((3,1))
policy = np.zeros((500,1))

# Get 500 steps from the trained model
# Get the best next step for each state by comparing all three state-action values.
# Store each step in policy 
for i in range(500):
    policy[i] = St

    x[0][0] = St
    x[0][1] = 0
    NextValue[0] = target_net(x).detach()
    print('val act 0',NextValue[0])
    
    x[0][1] = 1
    NextValue[1] = target_net(x).detach()
    print('val act 1',NextValue[1])
    
    x[0][1] = 2
    NextValue[2] = target_net(x).detach()
    print('val act 2',NextValue[2])
    
    nextAct = int(np.argmax(NextValue))
    print(nextAct)
    newState = next_states[St,nextAct]
    St = int(newState)
    
    

In [ ]:
# Plot the policy
thetaVals = np.zeros((100,1))
for i in range(100):
    
    st = policy[i]
    thetaVals[i],_ = get_values(st)
    
    
plt.plot(range(100), thetaVals)
draw_pendulum(thetaVals)
plt.xlabel("time")
plt.ylabel("theta$")
plt.show()

In [ ]:
def freependulum(state):
    '''
    To check dynamics of pendulum without control. 
    This function takes a state value from 0 to 2499 and displays SHM of pendulum with 0 force.  
    '''
    freeswing = np.zeros([100,1])
    freeswingtheta = np.zeros([100,1])
    for i in range(50):
        freeswing[i] = next_states[state,2]
        sat = int(freeswing[i])
        freeswingtheta[i],_ = get_values(state)


    plt.plot(range(100), freeswingtheta)
    draw_pendulum(freeswingtheta)
    plt.xlabel("time (ms)")
    plt.ylabel("theta")
    plt.show()

In [ ]:
# freependulum(1035)